In [ ]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [1]:
!pip install mlxtend --quiet
!pip install pandas numpy matplotlib --quiet

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
import pandas as pd
data = pd.read_csv('/content/drive/MyDrive/Groceries_dataset.csv')
print("Dữ liệu ban đầu: ", data.shape)
print("5 Bộ dữ liệu đầu tiên", data.head())

Dữ liệu ban đầu:  (38765, 3)
5 Bộ dữ liệu đầu tiên    Member_number        Date   itemDescription
0           1808  21-07-2015    tropical fruit
1           2552  05-01-2015        whole milk
2           2300  19-09-2015         pip fruit
3           1187  12-12-2015  other vegetables
4           3037  01-02-2015        whole milk


Làm sạch dữ liệu cơ bản

In [ ]:
import re
data['Date'] = pd.to_datetime(data['Date'], dayfirst = True, errors = 'coerce')
data = data.dropna(subset = ['Date'])
data['itemDescription'] = data['itemDescription'].astype(str)
data = data[data['itemDescription'].str.strip() != '']
def clean_item(s):
  s = str(s).lower().strip()
  s = re.sub(r'[/\\]', ' ', s)   # thay / hoặc \ bằng khoảng trắng
  s = re.sub(r'[^a-z0-9\s\-\&]', ' ', s) # chỉ giữ lại chữ, số, -, &, khoảng trắng
  s = re.sub(r'\s*-\s*', '-', s) # chuẩn hoá dấu gạch ngang
  s = re.sub(r'\s*&\s*', '&', s)# chuẩn hoá dấu &
  s = re.sub(r'\s+', ' ', s).strip() # loại bỏ khoảng trắng thừa
  return s
data['item_clean'] = data['itemDescription'].apply(clean_item)

Xây dựng mã giao dịch


In [ ]:
data['Member_number'] = data['Member_number'].astype(str)
data['TransID'] = data['Member_number'] + '_' + data['Date'].dt.strftime('%Y-%m-%d')

Gom các mặt hàng trong cùng giao dịch

In [ ]:
transactions = (
    data.groupby('TransID')['item_clean']
        .apply(lambda x: list(dict.fromkeys(x)))  # unique + giữ thứ tự xuất hiện
        .tolist()
)
print("Số giao dịch:", len(transactions))

Đếm và lọc mặt hàng hiếm

In [ ]:
from collections import Counter
cnt = Counter(it for t in transactions for it in t)
print("Số giao dịch ban đầu: ", len(transactions))
print("Top items: ", cnt.most_common(15))
min_count = 5
freq_items = {it for it, c in cnt.items() if c >= min_count}
transactions = [[it for it in t if it in freq_items] for t in transactions]
transactions = [t for t in transactions if len(t) >= 1]

Số giao dịch ban đầu:  14963
Top items:  [('whole milk', 2363), ('other vegetables', 1827), ('rolls buns', 1646), ('soda', 1453), ('yogurt', 1285), ('root vegetables', 1041), ('tropical fruit', 1014), ('bottled water', 908), ('sausage', 903), ('citrus fruit', 795), ('pastry', 774), ('pip fruit', 734), ('shopping bags', 712), ('canned beer', 702), ('bottled beer', 678)]


Chuyển sang dạng one-hot encoding chuẩn bị cho khai phá luật kết hợp

In [ ]:
from mlxtend.preprocessing import TransactionEncoder
te = TransactionEncoder()
te_ary = te.fit(transactions).transform(transactions, sparse=True)  # dùng sparse cho tiết kiệm RAM
df_onehot = pd.DataFrame.sparse.from_spmatrix(te_ary, columns=te.columns_)
print("Số mặt hàng (cột):", df_onehot.shape[1])
df_onehot.iloc[:5, :10]  # xem 5 giao dịch đầu, 10 cột đầu

One-hot saved, shape:  (14963, 163)


Chạy apriori

In [ ]:
from mlxtend.frequent_patterns import apriori

# NGƯỠNG BẠN CÓ THỂ ĐIỀU CHỈNH:
MIN_SUPPORT = 0.01   # 1% (tuỳ dữ liệu; nếu kết quả quá ít, giảm xuống 0.005; quá nhiều, tăng lên)
MAX_LEN     = 3      # xét tối đa 3 sản phẩm / tập (tăng nếu muốn)

frequent_itemsets = apriori(
    df_onehot,
    min_support=MIN_SUPPORT,
    use_colnames=True,
    max_len=MAX_LEN
).sort_values('support', ascending=False)

print("Số tập phổ biến tìm được:", frequent_itemsets.shape[0])
frequent_itemsets.head(10)

Số tập phổ biến tìm được:  69
Top 10 tâp phổ biến nhất: 
     support            itemsets
62  0.157923        (whole milk)
39  0.122101  (other vegetables)
45  0.110005        (rolls buns)
51  0.097106              (soda)
63  0.085879            (yogurt)
46  0.069572   (root vegetables)
56  0.067767    (tropical fruit)
4   0.060683     (bottled water)
48  0.060349           (sausage)
14  0.053131      (citrus fruit)
Đã lưu frequent_itemsets.csv


Sinh luật kết hợp từ các tập

In [ ]:
from mlxtend.frequent_patterns import association_rules

# Sinh luật từ frequent itemsets
rules = association_rules(
    frequent_itemsets,
    metric="confidence",
    min_threshold=0.1   # ngưỡng confidence tối thiểu ban đầu (để sinh nhiều luật); sẽ lọc sau
)

# Chuẩn hoá dạng hiển thị vế trái/phải
def frozenset_to_str(fs):
    return ", ".join(sorted(list(fs)))

rules['antecedents_str'] = rules['antecedents'].apply(frozenset_to_str)
rules['consequents_str'] = rules['consequents'].apply(frozenset_to_str)

# LỌC LUẬT MẠNH (tuỳ môn, thường lấy: support >= s0, confidence >= c0, lift > 1)
S0 = 0.01          # 1%
C0 = 0.3           # 30%
L0 = 1.2           # lift > 1.2
strong_rules = rules[(rules['support'] >= S0) &
                     (rules['confidence'] >= C0) &
                     (rules['lift'] > L0)].copy()

print("TỔNG SỐ LUẬT:", rules.shape[0])
print("SỐ LUẬT MẠNH (support>=%.3f, conf>=%.2f, lift>%.2f): %d" % (S0, C0, L0, strong_rules.shape[0]))

# Xem Top 20 luật mạnh theo lift
cols_show = ['antecedents_str','consequents_str','support','confidence','lift','leverage','conviction']
strong_rules.sort_values('lift', ascending=False).head(20)[cols_show]

Tổng số luật tạo ra:  4
Top 10 luật lift cao nhất
          antecedents   consequents   support  confidence      lift  \
3            (yogurt)  (whole milk)  0.011161    0.129961  0.822940   
1        (rolls buns)  (whole milk)  0.013968    0.126974  0.804028   
0  (other vegetables)  (whole milk)  0.014837    0.121511  0.769430   
2              (soda)  (whole milk)  0.011629    0.119752  0.758296   

   antecedent_len  
3               1  
1               1  
0               1  
2               1  
Đã lưu all_association_rules.csv
